## KNN implementation for Breast Cancer Classification
**Mamello Maseko(fill me :)) and Sandile Shongwe(1236067)**

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize
%xmode plain

Exception reporting mode: Plain


In [2]:
df = pd.read_csv('data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] #drop unnamed column
df_xtrain = df.drop(['id','diagnosis'], axis=1)
x_train = df_xtrain.values
#  Note: Makes M = 1, B=0
df['diagnosis'] = np.unique(df['diagnosis'], return_index=True, return_inverse=True)[2]
y_train = df['diagnosis'].values
x_train, y_train;

In [3]:
# #normalizing the data
# x_train = normalize(x_train, norm='l1', axis=1)
# print(x_train[0])

# mean = x_train.mean(axis=1)
# std = x_train.std(axis = 1)
# x_train = (x_train-mean[:,np.newaxis]) / std[:, np.newaxis]
#both normalization techniques seem to lower accuracy of the classifier

In [4]:
#splitting data set into training and testing sets using a 70 - 30 split  
length = len(x_train)
x_test = x_train[int(np.floor(length*0.7)+1): length  , :]
y_test = y_train[int(np.floor(length*0.7)+1): length]
x_train = x_train[0:int(np.floor(length*0.7))+1, :]
y_train = y_train[0:int(np.floor(length*0.7))+1]
y_train[y_train == 0] = -1;
y_test[y_test == 0] = -1;

In [5]:
##KNN using Euclidean distance
def KNN_E(x_train, y_train, query_point, K):
    dist = np.sqrt(np.sum((x_train - query_point)**2, axis = 1))
    idx = np.argsort(dist)
    s = np.sum(y_train[idx[np.arange(K)]])
    if(s > 0):
        return 1
    else:
        return -1

def KNN_M(x_train, y_train, query_point, K):
    dist = np.abs(np.sum((x_train - query_point), axis = 1))
    idx = np.argsort(dist)
    s = np.sum(y_train[idx[np.arange(K)]])
    if(s > 0.0):
        return 1
    else:
        return -1
# KNN training using Euclidean Distance    
def KNN_Elearn(x_train, y_train, x_test, y_test):
    error = 100000000
    min_k = 1000
    for K in range(1, 250):

        tmp = error
        diff_error = KNN_error_e(K, x_train, y_train, x_train, y_train) -  KNN_error_e(K, x_train, y_train,x_test, y_test)
        error = min(error, abs(diff_error))
        if tmp != error:
            min_k = K

    return min_k

def KNN_error_e(K, x_train, y_train, x_query, y_query):
    h = np.zeros((x_query.shape[0]))
    for i in range(x_query.shape[0]):
        h[i] = KNN_E(x_train, y_train,  x_query[i,:], K)
    e = np.sum(h != y_query*1.0)/(y_query.shape[0]*1.0)
    return e
# KNN training using Manhattan Distance
def KNN_Mlearn(x_train, y_train, x_test, y_test):
    error = 100000000
    min_k = 1000
    for K in range(1, 250):

        tmp = error
        diff_error = KNN_error_m(K, x_train, y_train, x_train, y_train) -  KNN_error_m(K, x_train, y_train,x_test, y_test)
        error = min(error, abs(diff_error))
        if tmp != error:
            min_k = K

    return min_k
def KNN_error_m(K, x_train, y_train, x_query, y_query):
    h = np.zeros((x_query.shape[0]))
    for i in range(x_query.shape[0]):
        h[i] = KNN_M(x_train, y_train,  x_query[i,:], K)
    e = np.sum(h != y_query*1.0)/(y_query.shape[0]*1.0)
    return e


In [6]:
opt_k_e = KNN_Elearn(x_train, y_train, x_test, y_test)
opt_k_m = KNN_Mlearn(x_train, y_train, x_test, y_test)

In [7]:
out_ye = [KNN_E(x_train, y_train,  x_test[i], opt_k_e) for i in range(170)]
out_ym = [KNN_E(x_train, y_train,  x_test[i], opt_k_m) for i in range(170)]

In [8]:
acc = np.sum(out_ye == y_test*1.0)/y_test.shape[0];
print('The accuracy using Euclidean Distance: {:.2f}'.format(acc*100))
acc = np.sum(out_ym == y_test*1.0)/y_test.shape[0];
print('The accuracy using Manhattan Distance: {:.2f}'.format(acc*100))

The accuracy using Euclidean Distance: 94.12
The accuracy using Manhattan Distance: 95.29
